# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
This notebook extracts and processes the data in the table of postcodes from the wikipedia page at: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

We ignore unused postcodes and merge all neighbourhoods with the same postcode into a single row, and assign the suburb name to the neighbourhood where one the neighbourhood has not been assigned a separate name.


### Import the packages we'll use

In [142]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML

### Download the wiki page and convert into an object tree

In [143]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

wiki_page = requests.get(url).text
    
soup = BeautifulSoup(wiki_page,'lxml')

### Find the table part of the page and extract the headers
3.1 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [144]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#print(My_table)

ths = My_table.findAll('th')
headers=[]
for th in ths:
    headers.append(th.text.strip())
headers

['Postcode', 'Borough', 'Neighbourhood']

### Now add the data
Find all of the table rows containing data and build up a dictionary of postcodes, boroughs and neighbourhoods, filtering and appending neighbourhoods to existing postcodes as we go.
* 3.2 Only process the cells that have an assigned borough. _Ignore cells with a borough that is Not assigned._
* 3.3 More than one neighborhood can exist in one postal code area. _For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table._
* 3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. _So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park._

In [145]:
# Find all of the table rows to process
rows = My_table.findAll('tr')
postcode_list={}
# lose the first row as that's the headers that we have already processed
rows=rows[1:]
for row in rows:
    tds = row.findAll('td')
    postcode = tds[0].text.strip()
    borough = tds[1].text.strip()
    neigh = tds[2].text.strip()
    
    #3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
    if (neigh == "Not assigned"):
        neigh = borough

    ## 3.2 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if (borough != 'Not assigned'):
        postcode_dict = {
            "Postcode": postcode, 
            "Borough": borough, 
            "Neighbourhood": neigh
        }
        ## 3.3 If we already have an entry for this postcode, then concatenate the neighbourhoods and updating the existing entry
        if (postcode in postcode_list):
            concatenated_neigh = postcode_list[postcode]["Neighbourhood"] + ", " + neigh
            postcode_dict.update({"Neighbourhood": concatenated_neigh})
            postcode_list[postcode].update(postcode_dict)
        else:
            postcode_list[postcode] = postcode_dict

### Convert our dictionary of postcode data into a dataframe
Show the first 12 entries.

In [146]:
df = pd.DataFrame(list(postcode_list.values()), columns=headers)
df.set_index('Postcode')
df.head(12)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M5E,Downtown Toronto,Berczy Park
2,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
3,M4J,East York,East Toronto
4,M6N,York,"The Junction North, Runnymede"
5,M1S,Scarborough,Agincourt
6,M4P,Central Toronto,Davisville North
7,M4N,Central Toronto,Lawrence Park
8,M7R,Mississauga,Canada Post Gateway Processing Centre
9,M5C,Downtown Toronto,St. James Town


# Now check the shape of the data frame
3.6 In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [147]:
print("The dataframe has {} rows.".format(df.shape[0]))

The dataframe has 103 rows.


### Obtain co-ordinates for the suburbs
* 4.2 Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [148]:
#use geocoder or csv

import csv

csv_url = 'https://cocl.us/Geospatial_data'

with requests.Session() as s:
    download = s.get(csv_url)

    decoded_content = download.content.decode('utf-8')

    reader = csv.reader(decoded_content.splitlines(), delimiter=',')
    csv_data = list(reader)
            
latlong_df=pd.DataFrame(csv_data[1:], columns=['Postcode', 'Latitude', 'Longitude'])
latlong_df.set_index('Postcode')
latlong_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.8066863,-79.1943534
1,M1C,43.7845351,-79.1604971
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.2169174
4,M1H,43.773136,-79.2394761


### Update our dataframe with the co-ordinates of each postcode
e.g. 'M4N' should have co-ordinates: ('43.7280205', '-79.3887901')

In [149]:
neighbourhoods = pd.merge(df, latlong_df, how='outer', on='Postcode')
#convert the lat/long strings to numeric
neighbourhoods[['Latitude', 'Longitude']] = neighbourhoods[['Latitude', 'Longitude']].apply(pd.to_numeric)
neighbourhoods.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
2,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
3,M4J,East York,East Toronto,43.685347,-79.338106
4,M6N,York,"The Junction North, Runnymede",43.673185,-79.487262
5,M1S,Scarborough,Agincourt,43.794200,-79.262029
6,M4P,Central Toronto,Davisville North,43.712751,-79.390197
7,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
8,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819
9,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


### Explore and cluster the neighborhoods in Toronto
Explore and cluster the neighborhoods in Toronto. 
You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:
* to add enough Markdown cells to explain what you decided to do and to report any observations you make.
* to generate maps to visualize your neighborhoods and how they cluster together.

#### Use geopy library to get the latitude and longitude values of Toronto.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>t_explorer</em>, as shown below.

Import all the libraries we will use

In [150]:
#Import Libraries
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed this yet
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [151]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [152]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [153]:
### Delve a bit deeper and look at boroughs with 'Toronto' in their name

In [154]:
### So let's slice the original dataframe and create a new dataframe of the 'Toronto' borough data.

In [155]:
subset_data = neighbourhoods[neighbourhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
subset_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763


Let's get the geographical coordinates of DownTown Toronto.

In [156]:
address = 'DownTown Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of DownTown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of DownTown Toronto are 43.655115, -79.380219.


As we did with all of Toronto, let's visualize the neighborhoods in Downtown Toronto.

In [157]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(subset_data['Latitude'], subset_data['Longitude'], subset_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [158]:
CLIENT_ID = 'SSH54FSXROYXILRRQFG4PQIFYZBKZ5PV0OEULGIJJS2TLBJN' # your Foursquare ID
CLIENT_SECRET = 'CWXT4OKPBGEKWULOJJ0E2FSFU5MDQJF5YDRD5ETB3DMOOOO0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#print('Credentials:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [159]:
subset_data.loc[0, 'Neighbourhood']
neighborhood_latitude = subset_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = subset_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = subset_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Berczy Park are 43.644770799999996, -79.3733064.


#### Now, let's get the top 100 venues that are within a radius of 500 meters.

First, let's create the GET request URL.

In [160]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Send the GET request and examine the results

In [161]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9be0496a60715342616b3b'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4aeb719af964a52020c221e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_liquor_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d186941735',
         'name': 'Liquor Store',
         'pluralName': 'Liquor Stores',
         'primary': True,
         'shortName': 'Liquor Store'}],
       'id': '4aeb719af964a52020c221e3',
       'location': {'address': '2 Cooper St',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Queens Quay E',
        'distance': 215,
        'formattedAddress': ['2 Cooper St (at Queens Quay E)',
         'Toronto ON M5E 1A4',
         'Canada'],
        'labeledLa

We know that all the information is in the *items* key. First, let's define a **get_category_type** function.

In [162]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [163]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,LCBO,Liquor Store,43.642944,-79.372440
1,The Keg Steakhouse + Bar,Steakhouse,43.646676,-79.374822
2,Sony Centre for the Performing Arts,Concert Hall,43.646292,-79.376022
3,Hockey Hall Of Fame (Hockey Hall of Fame),Museum,43.646974,-79.377323
4,Fresh On Front,Vegetarian / Vegan Restaurant,43.647888,-79.374586


## 2. Explore Neighborhoods in Downtown Toronto

#### Create a function to repeat the same process to all the neighborhoods in Downtown Toronto

In [164]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Ceate a new dataframe called *toronto_venues* by running this function against each neighbourhood

In [165]:
toronto_venues = getNearbyVenues(names=subset_data['Neighbourhood'],
                                   latitudes=subset_data['Latitude'],
                                   longitudes=subset_data['Longitude']
                                  )

toronto_venues.head(5)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
1,Berczy Park,43.644771,-79.373306,The Keg Steakhouse + Bar,43.646676,-79.374822,Steakhouse
2,Berczy Park,43.644771,-79.373306,Sony Centre for the Performing Arts,43.646292,-79.376022,Concert Hall
3,Berczy Park,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum
4,Berczy Park,43.644771,-79.373306,Fresh On Front,43.647888,-79.374586,Vegetarian / Vegan Restaurant


Let's check how many venues were returned for each neighborhood

In [166]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",12,12,12,12,12,12
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,81,81,81,81,81,81
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


#### Let's find out how many unique categories can be curated from all the returned venues

In [167]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 unique categories.


## 3. Analyze Each Neighborhood

In [170]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [171]:
toronto_onehot.shape

(1693, 238)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [172]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.01,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,0.0,0.0,0.010000,0.01,0.0,0.03,0.0,0.000000,0.000000,0.0,0.000000,0.030000,0.0,0.040000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.020000,0.000000,0.0,0.01,0.03,0.010000,0.0,0.000000,0.040000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.010000,0.010000,0.060000,0.0,0.0,0.0,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.0,0.000000,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.000000,0.000000,0.000000,0.0

In [173]:
toronto_grouped.shape

(38, 238)

#### Let's put each neighbourhood's top 10 most common venues into a *pandas* dataframe
Let's write a function to sort the venues in descending order.

In [174]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [175]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,Gym,Asian Restaurant,Burger Joint,Hotel,American Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Pub,Restaurant,Cheese Shop,Farmers Market,Café,Steakhouse,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Burrito Place,Nightclub,Bar,Stadium,Furniture / Home Store,Climbing Gym,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Recording Studio,Smoke Shop,Skate Park,Brewery,Burrito Place,Butcher,Restaurant,Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Airport Terminal,Boutique,Boat or Ferry,Sculpture Garden


## 4. Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.

In [176]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood.

In [177]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = subset_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Pub,Restaurant,Cheese Shop,Farmers Market,Café,Steakhouse,Bakery
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Sandwich Place,Breakfast Spot,Park,Gym,Grocery Store,Hotel,Food & Drink Shop,Dance Studio,Clothing Store,Burger Joint
2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Hotel,Café,Breakfast Spot,Clothing Store,Gastropub,Cosmetics Shop,Bakery,BBQ Joint
4,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,0,Mexican Restaurant,Café,Bar,Flea Market,Gastropub,Music Venue,Diner,Cajun / Creole Restaurant,Sandwich Place,Fast Food Restaurant


Finally, let's visualize the resulting clusters

In [178]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# Hide secrets cell?
# replicate the same analysis we did to the New York City data. It is up to you.

# Just make sure:

# to add enough Markdown cells to explain what you decided to do and to report any observations you make.
# to generate maps to visualize your neighborhoods and how they cluster together.